In [15]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as wg

In [16]:
# define mixing model
def mixer(x,t,Tf,Caf):
    # Inputs (2):
    # Tf = Feed Temperature (K)
    # Caf = Feed Concentration (mol/m^3)
    # States (2):
    # Concentration of A (mol/m^3)
    Ca = x[0]
    # Parameters:
    # Volumetric Flowrate (m^3/hr)
    q = 100
    # Volume of CSTR (m^3)
    V = 100
    # Calculate concentration derivative
    dCadt = q/V*(Caf - Ca)
    return dCadt

In [17]:
# Simulate mixer
def PID(Kc,tauI,tauD):
    
    # Initial Condition
    Ca0 = 0.0
    # Feed Temperature (K)
    Tf = 300 
    # Time Interval (min)
    t = np.linspace(0,10,100)

    # Feed Concentration (mol/m^3)
    Caf = np.zeros(len(t))

    # Storage for plotting
    Ca = np.zeros(len(t))
    Ca[0] = Ca0
    e = np.zeros(len(t))
    
    # PID controller 
    SP = np.ones(len(t))*1.5
    SP[50:] = 1.0
    Kp = 0.0
    Kd = 0.0
    
    for i in range(len(t)-1):
        e[i] = SP[i] - Ca0
        if i>=1:
            Kd = (Ca[i]-Ca[i-1])/(t[i+1]-t[i])
        Kp = Kp + e[i]*(t[i+1]-t[i])
        Caf[i] = Caf[0] + (Kc*e[i]) + (Kc/tauI*Kp) - (Kc*tauD*Kd)
        y = odeint(mixer,Ca0,[t[i],t[i+1]],args=(Tf,Caf[i]))
        Ca0 = y[1]
        Ca[i+1] = Ca0
        
    # Plot the results
    plt.plot(t,Ca,'r-',linewidth=3,label='Concentration')
    plt.plot(t,e,'g--',linewidth=3,label='Error')
    plt.plot(t,SP,'k--',linewidth=1,label='Set Point')
    plt.plot(t,Caf,'b:',linewidth=3,label='Inlet Concentration')
    plt.ylabel('Ca (mol/L)')
    plt.legend(loc='best')
    plt.xlabel('Time (hr)')
    plt.show()    


In [18]:
# Interactive interface to control gains
Kc_slide = wg.FloatSlider(value=3.5,min=-0.2,max=4.0,step=0.05)
tauI_slide = wg.FloatSlider(value=0.8,min=-0.2,max=1.5,step=0.05)
tauD_slide = wg.FloatSlider(value=0.1,min=-0.2,max=1.5,step=0.05)
wg.interact(PID, Kc=Kc_slide, tauI=tauI_slide, tauD=tauD_slide)

A Jupyter Widget

<function __main__.PID(Kc, tauI, tauD)>